<a href="https://colab.research.google.com/github/Abdul-Baatinul-Haad/Data-Collection-and-Curation/blob/main/BDAT1008_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Abdul-Baatinul-Haad Yusuf, BDAT1008 Assignment1

In [2]:
!apt-get update #update linux
!apt-get install openjdk-8-jdk-headless -qq > /dev/null #download and install openjdk
!wget -q http://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz #download spark binary (gunzip)
!tar xf spark-3.2.1-bin-hadoop2.7.tgz #extract the spark package
!pip install -q findspark #install the findspark package

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://archive

In [3]:
#set environment variables
import os 
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop2.7"


In [4]:
import findspark
findspark.init()
from pyspark import SparkContext

sc = SparkContext.getOrCreate()
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [7]:
# import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 

# File location and type
path = '/content/covid_19_202001_202007.csv'
file_type = "csv"

# CSV options, Using Inferred Schema
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# Import csv. The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(path)

df.head() #first row object

Row(Province/State=None, Country/Region='Afghanistan', Lat=33.93911, Long=67.709953, Date='2020-01-22', Confirmed=0, Deaths=0, Recovered=0, Active=0, WHO Region='Eastern Mediterranean')

In [8]:
df

DataFrame[Province/State: string, Country/Region: string, Lat: double, Long: double, Date: string, Confirmed: int, Deaths: int, Recovered: int, Active: int, WHO Region: string]

In [9]:
#define schema before importing
from pyspark.sql.types import StructType, IntegerType, DoubleType, StringType
schema = StructType()
schema.add("Province/State", "string", True)
schema.add("Country/Region", "string", True)
schema.add("Lat", "double", True)
schema.add("Long", "double", True)
schema.add("Date", "string", True)
schema.add("Confirmed", "integer", True)
schema.add("Deaths", "integer", True)
schema.add("Recovered", "integer", True)
schema.add("Actve", "integer", True)
schema.add("WHO Region", "string", True)
df = spark.read.format(file_type) \
  .option("schema", schema) \
  .option("header", True) \
  .option("sep", delimiter) \
  .load(path)

df.head()

Row(Province/State=None, Country/Region='Afghanistan', Lat='33.93911', Long='67.709953', Date='2020-01-22', Confirmed='0', Deaths='0', Recovered='0', Active='0', WHO Region='Eastern Mediterranean')

In [10]:
#Summary Statistics
df.describe().show()

+-------+--------------+--------------+------------------+------------------+----------+------------------+-----------------+-----------------+-----------------+---------------+
|summary|Province/State|Country/Region|               Lat|              Long|      Date|         Confirmed|           Deaths|        Recovered|           Active|     WHO Region|
+-------+--------------+--------------+------------------+------------------+----------+------------------+-----------------+-----------------+-----------------+---------------+
|  count|         14664|         49068|             49068|             49068|     49068|             49068|            49068|            49068|            49068|          49068|
|   mean|          null|          null|21.433730459769688|23.528236452106245|      null| 16884.90425531915|884.1791595337083|7915.713479253282| 8085.01161653216|           null|
| stddev|          null|          null| 24.95031982606509|  70.4427397445028|      null|127300.20527228026|631

In [11]:
#line chart to show deaths of top 10 countries suffering from covid-19
# first convert to pandas dataframe
df_pd = df.toPandas()
print(df_pd.dtypes)

Province/State    object
Country/Region    object
Lat               object
Long              object
Date              object
Confirmed         object
Deaths            object
Recovered         object
Active            object
WHO Region        object
dtype: object


In [21]:
from datetime import datetime
import pandas as pd
#Change to correct types
df_pd = df_pd.astype({"Province/State": str, "Country/Region" : str, "Lat" : float, "Long" : float, "Date" : str, "Confirmed" : int, "Deaths" : int, "Recovered" : int, "Active" : int, "WHO Region" : str})
df_pd['Date'] = pd.to_datetime(df_pd['Date'])
display(df_pd)

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active,WHO Region
0,None,Afghanistan,33.939110,67.709953,2020-01-22,0,0,0,0,Eastern Mediterranean
1,None,Albania,41.153300,20.168300,2020-01-22,0,0,0,0,Europe
2,None,Algeria,28.033900,1.659600,2020-01-22,0,0,0,0,Africa
3,None,Andorra,42.506300,1.521800,2020-01-22,0,0,0,0,Europe
4,None,Angola,-11.202700,17.873900,2020-01-22,0,0,0,0,Africa
...,...,...,...,...,...,...,...,...,...,...
49063,None,Sao Tome and Principe,0.186400,6.613100,2020-07-27,865,14,734,117,Africa
49064,None,Yemen,15.552727,48.516388,2020-07-27,1691,483,833,375,Eastern Mediterranean
49065,None,Comoros,-11.645500,43.333300,2020-07-27,354,7,328,19,Africa
49066,None,Tajikistan,38.861000,71.276100,2020-07-27,7235,60,6028,1147,Europe


In [13]:
#get top10 countries
import numpy as np
top10 = df_pd.sort_values(['Confirmed'],ascending=False).drop_duplicates(['Country/Region']).reset_index(drop=False)
top10 = top10.nlargest(10, 'Confirmed')
top10.index+=1
top10

,index,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active,WHO Region
1,49030,None,US,40.000000,-100.000000,2020-07-27,4290259,148011,1325804,2816444,Americas
2,48835,None,Brazil,-14.235000,-51.925300,2020-07-27,2442375,87618,1846641,508116,Americas
3,48936,None,India,20.593684,78.962880,2020-07-27,1480073,33408,951166,495499,South-East Asia
4,48992,None,Russia,61.524010,105.318756,2020-07-27,816680,13334,602249,201097,Europe
5,49005,None,South Africa,-30.559500,22.937500,2020-07-27,452529,7067,274925,170537,Africa
6,48963,None,Mexico,23.634500,-102.552800,2020-07-27,395489,44022,303810,47657,Americas
7,48986,None,Peru,-9.190000,-75.015200,2020-07-27,389717,18418,272547,98752,Americas
8,48854,None,Chile,-35.675100,-71.543000,2020-07-27,347923,9187,319954,18782,Americas
9,49028,None,United Kingdom,55.378100,-3.436000,2020-07-27,300111,45759,0,254352,Europe
10,48938,None,Iran,32.427908,53.688046,2020-07-27,293606,15912,255144,22550,Eastern Mediterranean


In [14]:
#Plot line chart for question 5 using plotly
import plotly.express as px
fig = px.line(top10, x="Country/Region", y="Deaths", title = 'Top 10 Countries Suffering From Covid-19')
fig.show()

In [15]:
#Question 6
# define start and end dates, and get records between them
start_date = '2020-01-22'
end_date = '2020-07-27'
mask = (df_pd['Date'] > start_date) & (df_pd['Date'] <= end_date)
df6 = df_pd.loc[mask]
display(df6)

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active,WHO Region
261,None,Afghanistan,33.939110,67.709953,2020-01-23,0,0,0,0,Eastern Mediterranean
262,None,Albania,41.153300,20.168300,2020-01-23,0,0,0,0,Europe
263,None,Algeria,28.033900,1.659600,2020-01-23,0,0,0,0,Africa
264,None,Andorra,42.506300,1.521800,2020-01-23,0,0,0,0,Europe
265,None,Angola,-11.202700,17.873900,2020-01-23,0,0,0,0,Africa
...,...,...,...,...,...,...,...,...,...,...
49063,None,Sao Tome and Principe,0.186400,6.613100,2020-07-27,865,14,734,117,Africa
49064,None,Yemen,15.552727,48.516388,2020-07-27,1691,483,833,375,Eastern Mediterranean
49065,None,Comoros,-11.645500,43.333300,2020-07-27,354,7,328,19,Africa
49066,None,Tajikistan,38.861000,71.276100,2020-07-27,7235,60,6028,1147,Europe


In [16]:
#calculate rocovery rate and add it as a column
def r_rate(row):  
    if row['Recovered'] == 0 or row['Confirmed'] == 0:
        return 0
    else:
      return (row['Recovered'] / row['Confirmed']) * 100

df6['Recovery Rate'] = df6.apply(lambda row: r_rate(row), axis=1)
df6

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active,WHO Region,Recovery Rate
261,None,Afghanistan,33.939110,67.709953,2020-01-23,0,0,0,0,Eastern Mediterranean,0.000000
262,None,Albania,41.153300,20.168300,2020-01-23,0,0,0,0,Europe,0.000000
263,None,Algeria,28.033900,1.659600,2020-01-23,0,0,0,0,Africa,0.000000
264,None,Andorra,42.506300,1.521800,2020-01-23,0,0,0,0,Europe,0.000000
265,None,Angola,-11.202700,17.873900,2020-01-23,0,0,0,0,Africa,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
49063,None,Sao Tome and Principe,0.186400,6.613100,2020-07-27,865,14,734,117,Africa,84.855491
49064,None,Yemen,15.552727,48.516388,2020-07-27,1691,483,833,375,Eastern Mediterranean,49.260792
49065,None,Comoros,-11.645500,43.333300,2020-07-27,354,7,328,19,Africa,92.655367
49066,None,Tajikistan,38.861000,71.276100,2020-07-27,7235,60,6028,1147,Europe,83.317208


In [17]:
#plot recovery rate from 22-jan-2020 to 27-july-2020
fig = px.histogram(df6, x="Date", y="Recovery Rate", title = 'Recovery Rate From The Period of 22-Jan to 27-July in 2020')
fig.show()

In [18]:
#top 5 countries with relatively higher recovery rate
#first sort with descending recovery rate values and eleinate dulicate countries
#then use nlargest to get the top n rows with highest recovery rate
top5 = df6.sort_values(['Recovery Rate'],ascending=False).drop_duplicates(['Country/Region']).reset_index(drop=False)
top5 = top5.nlargest(5, 'Recovery Rate')
top5.index+=1
top5['Country/Region']

1                   Uganda
2                    China
3               Seychelles
4                Australia
5    Saint Kitts and Nevis
Name: Country/Region, dtype: object

In [24]:
#question 7
#first get the records for 2020-07-27 / 27 july, 2020
mask2 = (df6['Date'] == '2020-07-27')
df7 = df6.loc[mask2]

#then we perform the groupby and aggregate function using pandas.core.groupby.DataFramGroupBy.aggregate
#the columns to perform aggregations this on are Confirmed, Deaths, Recovered and Active, and the column to group by is (WHO Region)
#The aggregations i will make are: sum, min, max and mean
q7 = df7.groupby('WHO Region').agg({"Confirmed" : [min, max, sum, np.mean], "Deaths" : [min, max, sum, np.mean],
                                    "Recovered" : [min, max, sum, np.mean], "Active" : [min, max, sum, np.mean]})
q7

Confirmed                                  Deaths  \
                            min      max      sum           mean    min   
WHO Region                                                                
Africa                       10   452529   723207   15066.812500      0   
Americas                      5  4290259  8839286  192158.391304      0   
Eastern Mediterranean       674   293606  1490744   67761.090909     11   
Europe                        3   816680  3299523   41244.037500      0   
South-East Asia              24  1480073  1835297  183529.700000      0   
Western Pacific               1    82040   292428    5316.872727      0   

                                                   Recovered           \
                          max     sum         mean       min      max   
WHO Region                                                              
Africa                   7067   12223   254.645833         0   274925   
Americas               148011  342732  7450.695652         0  1846641   
Eastern Mediterranean   15912   38339  1742.681818         0   255144   
Europe                  45759  211144  2639.300000         0   602249   
South-East Asia         33408   41349  4134.900000         0   951166   
Western Pacific          4512    8249   149.981818         0    63623   

                                              Active                    \
                           sum           mean    min      max      sum   
WHO Region                                                               
Africa                  440645    9180.104167      1   170537   270339   
Americas               4468616   97143.826087      0  2816444  4027938   
Eastern Mediterranean  1201400   54609.090909     24    52992   251005   
Europe                 1993723   24921.537500      0   254352  1094656   
South-East Asia        1156933  115693.300000     13   495499   637015   
Western Pacific         206770    3759.454545      0    53649    77409   

                                     
                               mean  
WHO Region                           
Africa                  5632.062500  
Americas               87563.869565  
Eastern Mediterranean  11409.318182  
Europe                 13683.200000  
South-East Asia        63701.500000  
Western Pacific         1407.436364

In [30]:
#question 8 
#first get all records for the 3rd month after the when first case was confirmed
#the min date, which is the date of the first confirmed case is 2020-01-22.
#to get correct date 3 months after 2020-01-22 i will use datetime library to import date and also dateutil.relativedelta to import relativedelta
from datetime import date
from dateutil.relativedelta import relativedelta
maindate = date(2020,1,22) + relativedelta (months=+3)
print(maindate)

2020-04-22


In [34]:
#so the maindate is 2020-04-22
#now you can get the records easily
mask3 = (df6['Date'] == '2020-04-22')
df8 = df6.loc[mask3]
df8

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active,WHO Region,Recovery Rate
23751,None,Afghanistan,33.939110,67.709953,2020-04-22,1176,40,166,970,Eastern Mediterranean,14.115646
23752,None,Albania,41.153300,20.168300,2020-04-22,634,27,356,251,Europe,56.151420
23753,None,Algeria,28.033900,1.659600,2020-04-22,2910,402,1204,1304,Africa,41.374570
23754,None,Andorra,42.506300,1.521800,2020-04-22,723,37,309,377,Europe,42.738589
23755,None,Angola,-11.202700,17.873900,2020-04-22,25,2,6,17,Africa,24.000000
...,...,...,...,...,...,...,...,...,...,...,...
24007,None,Sao Tome and Principe,0.186400,6.613100,2020-04-22,4,0,0,4,Africa,0.000000
24008,None,Yemen,15.552727,48.516388,2020-04-22,1,0,0,1,Eastern Mediterranean,0.000000
24009,None,Comoros,-11.645500,43.333300,2020-04-22,0,0,0,0,Africa,0.000000
24010,None,Tajikistan,38.861000,71.276100,2020-04-22,0,0,0,0,Europe,0.000000


In [35]:
#finaly, we order by descending recovery rate to get the top 5 countries
q8top5 = df8.sort_values(['Recovery Rate'],ascending=False).drop_duplicates(['Country/Region']).reset_index(drop=False)
q8top5 = q8top5.nlargest(5, 'Recovery Rate')
q8top5.index+=1
q8top5['Country/Region']

1        Greenland
2            China
3           France
4      Saint Lucia
5    Liechtenstein
Name: Country/Region, dtype: object